In [74]:
import onnx
from onnx.shape_inference import infer_shapes, infer_shapes_path
from onnx import helper
onnx.__version__

'1.12.0'

In [90]:
from audioop import mul


model = onnx.load("tinyyolo_infershapes.onnx")
mulnodes = []
for index,n in enumerate(model.graph.node):
    if "Mul_" in n.name:
        mulnodes.append(index)

mul_node = model.graph.node[mulnodes[0]]
in_tensors = mul_node.input
out_tensor = mul_node.output
in_value_info = []
out_value_info = []

for x in model.graph.value_info:
    if in_tensors[0] in x.name or in_tensors[1] in x.name:
        in_value_info.append(x)
    elif out_tensor[0] in x.name:
        out_value_info.append(x)

In [95]:

# TODO: Make a graph from the node that I have extracted...
graph = helper.make_graph(
    nodes=[mul_node],
    name="simple_graph",
    inputs=[in_value_info[0], in_value_info[1]],
    # initializer = [in_value_info[1]],
    outputs=[out_value_info[0]],
)

onnx_model = helper.make_model(graph, producer_name="multiply-model")
onnx.save(onnx_model, 'multiply_model_split.onnx')

In [110]:
import onnxruntime as ort
import numpy as np
in1 = np.random.random([1,3,512,512]).astype(np.float32)
b = np.random.random([1]).astype(np.float32)
# out_ort = np.random.random([1,16,512,512]).astype(np.float32)
output = in1*b

input_dict = {"278": in1, "279": b}
sess = ort.InferenceSession(onnx_model.SerializeToString())
out_ort = sess.run(None,input_dict)

In [111]:
np.allclose(out_ort,output)

True